# Operation of a cooled exothermic CSTR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.optimize import fsolve

In [ ]:
# Data
Fo = 40       # Feed flow rate, ft^3/h
F = 40        # Output flow rate, ft^/h
Vr = 48       # Reactor volume, ft^3
Cao = 0.55    # Initial concentration of A, lbmol/ft^3
Cp = 0.75     # Heat capacity, btu/lb/R
rho = 50      # Fluid density, lb/ft^3
R = 1.9872    # Universal gas constant, bu/lbmol/R
To = 530      # Initial temperature, R

# Kinetic data
E = 30000     # Activation energy, btu/lbmol
ko = 7.08e10  # Preexponential factor, 1/h
DHr = -30000  # Reaction heat, btu/lbmol

# cooling jacket
Fj = 49.9      # Cooling water flow rate, ft^/h
Tjo = 525      # Inlet temperature, R
Cpj = 1        # Heat capacity, btu/lb/R
rhoj = 62.3    # Water density, lb/ft^3
U = 150        # overall heat transfer coefficient, btu/h/ft^2/R
A = 250        # heat transfer area, ft^2

In [ ]:
 def systemCST(x, p):

    U, A, E, ko, Fo, F, R, Cao, Vr, rho, Cp, To, \
    DHr, rhoj, Cpj, Fj, Tjo = p

    k = ko * np.exp(-E/R/x);
    Tj = (rhoj * Cpj * Fj * Tjo+ U*A*x)/(rhoj*Cpj*Fj+U*A);
    q = U*A*(x-Tj);

    cA = Fo*Cao/(F+Vr*k);
    HX = - DHr*Vr*k*cA;
    g = rho*Cp*(Fo*To-F*x) + HX - q;

    return [g, cA, HX, Tj]

In [ ]:
param = np.array([U, A, E, ko, Fo, F, R, Cao, Vr, rho, Cp, To, 
                  DHr, rhoj, Cpj, Fj, Tjo])

T = fsolve(@(x)f_003_04(x,param), To, options);
[~,CA, HX, Tj] = f_003_04(T,param);

fprintf('La concentracion final de A es %1.3f lbmol/ft^3\n', CA);
fprintf('La temperatura de reaccion %1.2f R\n', T);
fprintf('La temperatura final del refrigerante %1.2f R\n', Tj);

In [ ]:
%% Otra forma de visualizar los estados estacionarios
Tini = 500; 
Tfin = 700;

vT = linspace(Tini,Tfin)';
vSS = zeros(length(vT));

% Modelo matematico
vk = ko * exp(-E/R./vT);

% Output jacket temperature
vTj = (rhoj * Cpj * Fj * Tjo + U * A .* vT)/(rhoj * Cpj * Fj + U * A);

% Heat transfer between jacket and reactor
vq = U * A .* (vTj - vT);

% Final concentration of A
vcA = Fo * Cao ./ ( F + Vr .* vk);

% La funcion HPR representa el flujo de calor generado por reaccion
vHPR = - DHr * Vr .* vk .* vcA;

% La funcion HWR representa la rapidez del calor removido
vHWR = rho * Cp * (F .* vT - Fo * To) - vq;

vBE = vHPR - vHWR;

In [ ]:
figura = figure;
axes1 = axes('Parent',figura,'YTick',[500 550 600 650 700],'YGrid','on',...
    'XGrid','on',...
    'FontSize',12);
% ylim(axes1,[500 700]);
box(axes1,'on');
hold(axes1,'all');

plot(vBE,vT,'LineWidth',2);
plot(vSS,vT,'LineWidth',2,'LineStyle','--');
xlabel('Energy balance function');
ylabel('Reaction temperature (R)');

matlab2tikz('img/003_04_001.tikz','height','\figureheight','width',...
           '\figurewidth','showInfo', false);

%% Calculo de los distintos estados estacionarios
%
% Para calcular los diversos estados estacionarios, se escoge el rango de
% Temperatura finales de la reaccion y con estos valores se calcula la
% Concentracion de A final, la Temperatura del fluido refrigerante y luego
% se resuelve el balance de energia del reactor.
% Cuando el balance de energia sea igual a cero entonces estaras en un
% estado estacionario.
%
% Para calcular los valores exactos del estado estacionario

fcA = zeros(3,1);
fHX = zeros(3,1);
Tss = zeros(3,1);
guess = [540 590 670];

for w=1:length(guess)
    Tss(w) = fsolve(@(x)f_003_04(x,param),guess(w),options);
    [~,CA,HX,~] = f_003_04(Tss(w),param);
    fcA(w) = CA;
    fHX(w) = HX;
end

fprintf(['Se alcanza los siguientes estados estacionarios:\n\n'...
'   T (R)      C_A (lbmol/ft^3)\n']);
fprintf('  %1.2f         %1.3f \n', [Tss, fcA]');

%% Grafico para visualizar los estados estacionarios tradicional
% Esta variable contiene la curva HPR y HWR
%

figura2 = figure;
axes2 = axes('Parent',figura2,'XTick',[500 550 600 650 700],'YGrid','on',...
    'XGrid','on',...
    'FontSize',12);
% ylim(axes2,[0 8]*10^5);
box(axes2,'on');
hold(axes2,'all');

plot(vT,vHPR,'LineWidth',2,'LineStyle','--','DisplayName','HPR');
plot(vT,vHWR,'LineWidth',2,'DisplayName','HWR');
plot(Tss(1),fHX(1),'MarkerSize',6,'Marker','o','LineWidth',2,...
    'LineStyle','none',...
    'MarkerFaceColor',[0.235294118523598 0.235294118523598 0.235294118523598],...
    'Color',[0.235294118523598 0.235294118523598 0.235294118523598],...
    'DisplayName',sprintf('%1.0f R',Tss(1)));
plot(Tss(2),fHX(2),'MarkerSize',8,'Marker','o','LineWidth',1,...
    'LineStyle','none',...
    'DisplayName',sprintf('%1.0f R',Tss(2)),...
    'Color',[0 0 0]);
plot(Tss(3),fHX(3),'MarkerSize',6,'Marker','o','LineWidth',2,...
    'LineStyle','none',...
    'MarkerFaceColor',[0.235294118523598 0.235294118523598 0.235294118523598],...
    'Color',[0.235294118523598 0.235294118523598 0.235294118523598],...
    'DisplayName',sprintf('%1.0f R',Tss(3)));
ylabel('HPR or HWR (btu/h)','FontSize',12);
xlabel('Reaction temperature (R)','FontSize',12);

legend1 = legend(axes2,'show');
set(legend1,'Location','NorthWest','FontSize',10);

matlab2tikz('img/003_04_002.tikz','height','\figureheight','width',...
          '\figurewidth','showInfo', false);

